In [25]:
import numpy as np

import tvm
from tvm import relay
from tvm.relay.build_module import bind_params_by_name
from tvm.relay.dataflow_pattern import *
from tvm.relay.testing import run_opt_pass
from tvm.relay.collage import _ffi_api

# Pattern
pattern 匹配

In [3]:
K_ELEMWISE = 0
K_BROADCAST = 1

# Pattern
is_conv2d = is_op("nn.conv2d")(wildcard(), wildcard())
is_unary_elemwise = (wildcard().has_attr({"TOpPattern": K_ELEMWISE}))(wildcard())
reduction = is_op("add")(wildcard(), wildcard())
diamond = dominates(is_conv2d, is_unary_elemwise, reduction)

# Classic Diamond
inp = relay.var("input")
weight = relay.var("weight")
conv2d = relay.op.nn.conv2d(inp, weight)
relu = relay.op.nn.relu(conv2d)
relu = relay.op.nn.relu(relu)
leaky_relu = relay.op.nn.leaky_relu(conv2d, alpha=0)
out = relu + leaky_relu

# Check
assert diamond.match(out)

# Deeper Branch
inp = relay.var("input")
weight = relay.var("weight")
conv2d = relay.op.nn.conv2d(inp, weight)
relu = relay.op.nn.relu(conv2d)
relu = relay.op.nn.relu(relu)
relu = relay.op.tanh(relu)
leaky_relu = relay.op.nn.leaky_relu(conv2d, alpha=0)
out = relu + leaky_relu

# Check
assert diamond.match(out)

# Single Branch
inp = relay.var("input")
weight = relay.var("weight")
conv2d = relay.op.nn.conv2d(inp, weight)
relu = relay.op.nn.relu(conv2d)
relu = relay.op.nn.relu(relu)
tanh = relay.op.tanh(relu)
out = relu + tanh

# Check
assert diamond.match(out)

# Fuzzy path/nested Diamond
is_conv2d = is_op("nn.conv2d")(wildcard(), wildcard())
is_unary_elemwise = (wildcard().has_attr({"TOpPattern": K_ELEMWISE}))(wildcard()) | is_op(
    "add"
)(wildcard(), wildcard())
reduction = is_op("add")(wildcard(), wildcard())
diamond = dominates(is_conv2d, is_unary_elemwise, reduction)

inp = relay.var("input")
weight = relay.var("weight")
conv2d = relay.op.nn.conv2d(inp, weight)
relu = relay.op.nn.relu(conv2d)
relu = relu + relu
tanh = relay.op.tanh(relu)
leaky_relu = relay.op.nn.leaky_relu(conv2d, alpha=0)
out = tanh + leaky_relu

assert diamond.match(out)


应用组合pattern

例如，TVM默认fuse组装形式如下：
```
PrimitivePartitionRule
  OnlyValidPartitionRule
    CombinePartitionRule (with default TVM combiner rules)
      OpCallByKindPartitionRule
```

In [12]:
from tvm.relay import dataflow_pattern
rule_name = "my_partition_rule"
op_call_rule = dataflow_pattern.OpCallByKindPartitionRule(rule_name)

AttributeError: module 'tvm.relay.dataflow_pattern' has no attribute 'OpCallByKindPartitionRule'

In [15]:
pattern_table = tvm.relay.op.contrib.get_pattern_table("dnnl")
pattern_table[0]
    

('dnnl.qnn.conv2d',
 CallPatternNode(Op(cast), [AltPattern(CallPatternNode(Op(add), [AltPattern(CallPatternNode(Op(add), [AltPattern(CallPatternNode(Op(multiply), [CallPatternNode(Op(clip), [CallPatternNode(Op(multiply), [AltPattern(CallPatternNode(Op(add), [CallPatternNode(Op(cast), [CallPatternNode(Op(qnn.conv2d), [*, ConstantPattern(), Constant(0), Constant(0), ConstantPattern(), ConstantPattern()])]), ConstantPattern()]) | CallPatternNode(Op(cast), [CallPatternNode(Op(qnn.conv2d), [*, ConstantPattern(), Constant(0), Constant(0), ConstantPattern(), ConstantPattern()])])), ConstantPattern()])]), ConstantPattern()]) | CallPatternNode(Op(clip), [CallPatternNode(Op(multiply), [AltPattern(CallPatternNode(Op(add), [CallPatternNode(Op(cast), [CallPatternNode(Op(qnn.conv2d), [*, ConstantPattern(), Constant(0), Constant(0), ConstantPattern(), ConstantPattern()])]), ConstantPattern()]) | CallPatternNode(Op(cast), [CallPatternNode(Op(qnn.conv2d), [*, ConstantPattern(), Constant(0), Constant(0)

In [18]:
rule_name, dataflow_pattern = pattern_table[0]

In [19]:
print(rule_name)
print(dataflow_pattern)

dnnl.qnn.conv2d
CallPatternNode(Op(cast), [AltPattern(CallPatternNode(Op(add), [AltPattern(CallPatternNode(Op(add), [AltPattern(CallPatternNode(Op(multiply), [CallPatternNode(Op(clip), [CallPatternNode(Op(multiply), [AltPattern(CallPatternNode(Op(add), [CallPatternNode(Op(cast), [CallPatternNode(Op(qnn.conv2d), [*, ConstantPattern(), Constant(0), Constant(0), ConstantPattern(), ConstantPattern()])]), ConstantPattern()]) | CallPatternNode(Op(cast), [CallPatternNode(Op(qnn.conv2d), [*, ConstantPattern(), Constant(0), Constant(0), ConstantPattern(), ConstantPattern()])])), ConstantPattern()])]), ConstantPattern()]) | CallPatternNode(Op(clip), [CallPatternNode(Op(multiply), [AltPattern(CallPatternNode(Op(add), [CallPatternNode(Op(cast), [CallPatternNode(Op(qnn.conv2d), [*, ConstantPattern(), Constant(0), Constant(0), ConstantPattern(), ConstantPattern()])]), ConstantPattern()]) | CallPatternNode(Op(cast), [CallPatternNode(Op(qnn.conv2d), [*, ConstantPattern(), Constant(0), Constant(0), Con

In [29]:
def DefaultPatternPredicate():
    return True

In [31]:
c = _ffi_api.MakeLabelledDFPatternPartitionRuleWithPredicate("dnnl", rule_name, dataflow_pattern, DefaultPatternPredicate)

In [38]:
c.AllCandidates()

AttributeError: <class 'tvm.runtime.object.Object'> has no attribute AllCandidates

In [20]:
pattern_tuple = pattern_table[0]
relay.collage.collage.make_labelled_dfpattern_partition_rule_wrapper("DNNL", pattern_tuple)

TVMError: Traceback (most recent call last):
  0: 0x00005594086ad250
  File "/root/wang/tvm/include/tvm/runtime/packed_func.h", line 1731
TVMError: Function relay.collage.MakeLabelledDFPatternPartitionRule(0: basic_string<char>&, 1: runtime.String, 2: DFPatternNode, 3: (0: RelayExpr&) -> bool) -> relay.collage.PartitionRule expects 4 arguments, but 3 were provided.

['AltPattern',
 'AttrPattern',
 'CallPattern',
 'Callable',
 'ConstantPattern',
 'DFPattern',
 'DFPatternCallback',
 'DataTypePattern',
 'Dict',
 'DominatorPattern',
 'Expr',
 'ExprPattern',
 'FunctionPattern',
 'IfPattern',
 'LetPattern',
 'List',
 'Node',
 'Object',
 'Optional',
 'ShapePattern',
 'TupleGetItemPattern',
 'TuplePattern',
 'TypePattern',
 'VarPattern',
 'WildcardPattern',
 '_DFPatternCallback',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_ffi',
 '_ir',
 'astext',
 'dominates',
 'ffi',
 'get',
 'has_attr',
 'has_dtype',
 'has_shape',
 'has_type',
 'is_constant',
 'is_expr',
 'is_if',
 'is_let',
 'is_op',
 'is_tuple',
 'is_tuple_get_item',
 'is_var',
 'make_node',
 'match',
 'partition',
 'pretty_print',
 'register_df_node',
 'rewrite',
 'tvm',
 'tvm_ffi',
 'wildcard']

将融合模式与融合规则对齐

查看系统中已经注册的外部编译模式

In [6]:
tvm.relay.op.contrib.register._PATTERN_TABLES

{'arm_compute_lib': [('arm_compute_lib.conv2d',
   AltPattern(AltPattern(CallPatternNode(Op(nn.conv2d), [AltPattern(CallPatternNode(Op(nn.pad), [*, *]) | *), ConstantPattern()]) | CallPatternNode(Op(nn.bias_add), [CallPatternNode(Op(nn.conv2d), [AltPattern(CallPatternNode(Op(nn.pad), [*, *]) | *), ConstantPattern()]), ConstantPattern()])) | CallPatternNode(Op(nn.relu), [AltPattern(CallPatternNode(Op(nn.conv2d), [AltPattern(CallPatternNode(Op(nn.pad), [*, *]) | *), ConstantPattern()]) | CallPatternNode(Op(nn.bias_add), [CallPatternNode(Op(nn.conv2d), [AltPattern(CallPatternNode(Op(nn.pad), [*, *]) | *), ConstantPattern()]), ConstantPattern()]))])),
   <function tvm.relay.op.contrib.arm_compute_lib.arm_compute_lib_pattern_table.<locals>.check_conv(extract)>),
  ('arm_compute_lib.qnn_conv2d',
   CallPatternNode(Op(qnn.requantize), [AltPattern(AltPattern(CallPatternNode(Op(qnn.conv2d), [AltPattern(CallPatternNode(Op(nn.pad), [*, *]) | *), ConstantPattern(), ConstantPattern(), ConstantPatte